In [6]:
import pandas as pd
import os
import sys
sys.path.append('../src')

In [7]:
parent_directory = os.path.dirname(os.getcwd())

survey_directory = os.path.join(parent_directory, 'data', 'surveys', 'final')
surveys = ['ai_survey']
           
participant_mapping = {'22285773-a87e-4a68-a101-66bfd5cca2ed': 's004', 'a14a7c9d-937a-436c-84f1-cd979080ec0b': 's005', 
                       '80e2dc89-dc36-4c85-9028-07a45c0fa569': 's006', '323a83de-05e3-42dd-a975-4d8407f8bf35': 's009', 
                       'f86b2c13-eb4c-47a3-bb45-68b01088e20c': 's008', '518bb4c0-1eb3-454e-9c07-57b7f14a7bc2': 's007', 
                       '69383d84-3462-4b90-b37d-5bc10da0cf1f': 's010', '686fb9b6-423d-4130-82ed-dd94c65b7c71': 's011', 
                       '56a06a94-cbf4-4a97-9b11-c903cc44669d': 's012', '2a894bdd-3a63-4490-82a8-60aa04ff2891': 's013', 
                       '9d16d983-101a-464c-9056-d0526355d134': 's014', '641d914a-7477-480a-9951-c34376399207': 's015', 
                       '63ba282e-0156-45ae-8070-677e8b43be09': 's016', '3854412c-03ce-4be4-82b8-d548ed62c787': 's017', 
                       '06fbed4e-909b-4183-86d8-db2d58716fd5': 's018', '62041c7c-754a-48d4-b5bf-fbddae1c2c6a': 's019', 
                       '78dbfa30-3a9b-478d-982a-aa79d7d6936c': 's020', 'c7f491d8-640b-4dd4-bc11-f028d37d988e': 's021', 
                       '6240a1dc-00dd-482a-a21a-880f973a1064': 's022', '24c3882b-668d-476e-8b26-1f8be338e5e3': 's023', 
                       'd6d87b0e-93a0-4ed4-b167-35ca2247703b': 's024', '9ef17c2a-1a75-40cc-8d51-d543ddb013d6': 's025', 
                       '1eb00119-97bc-45c1-b454-b57b6dcdbddb': 's026', '970ea90c-1365-4622-aed6-31322a2dafa6': 's027', 
                       '5f1d309b-8acb-4d27-8260-8d86185fb1f6': 's028', '24cab759-3bb7-4a22-a68f-24fb8058ed0b': 's029', 
                       '51d6b3f3-753a-4d86-9172-72408101f876': 's030', '98587983-1da1-4fbe-bea0-bcfb58e6db57': 's031', 
                       '061446ad-baa8-4e5e-a3a9-e1d16ced69a5': 's032', 'a72d4062-e4a9-40aa-9a9a-89fca5aa6105': 's033', 
                       '81bf5c8a-11cc-4736-a601-9d9769f615e4': 's034', 'b1a4d269-ab36-4e74-8e00-c579bda18761': 's035', 
                       'd577300b-967e-45d7-a36e-e97f13434035': 's036', 'f1f83835-9d7f-4841-a1bb-3bff8f81f55f': 's037', 
                       'fd1311c6-da0b-44b6-91a9-fdcd08fc5a4b': 's038', 'fa74503f-f470-443d-9125-fbd66a3f2396': 's039', 
                       '315123d0-223b-4db6-b2a7-7f2ff51d907b': 's040', '3048cc74-83c1-445e-bb31-ec9069d6567e': 's041', 
                       '52ce857a-38b4-4ab4-a467-729e0686f2cc': 's042', '06e403ab-ebc3-4fde-965a-733f38a2792f': 's043'}

In [8]:
def correct_reverse_coding(row):
    """
    Correct the scoring of survey responses for reverse-coded questions.

    Parameters:
    - row: A pandas Series representing a single row of the DataFrame containing the survey data.

    Returns:
    - The corrected response value. If the response is NaN, returns None. 
      If the question is specifically 'ai_survey_5_options', applies reverse coding; 
      otherwise, returns the integer value of the response.
    """
    if pd.isna(row['response']):
        return None
    elif row['question_key'] == 'ai_survey_5_options':
        return 6 - int(row['response'])
    else:
        return int(row['response'])

In [9]:
survey_results = {}
for foldername in os.listdir(survey_directory):
    folder_path = os.path.join(survey_directory, foldername)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith('.csv'):
                if any(survey in filename for survey in surveys):
                    subject_id = filename.split('_')[0]
                    
                    if subject_id:
                        df = pd.read_csv(os.path.join(folder_path, filename))
                        df['corrected_value'] = df.apply(correct_reverse_coding, axis=1)

                        survey_results[subject_id] = {'data': df, 'average_score': df['corrected_value'].mean(), 
                                                      'total_score': df['corrected_value'].sum()}
                        for question in df['text']:
                            survey_results[subject_id][question] = df.loc[df['text'] == question, 'corrected_value'].values[0]

%store survey_results

Stored 'survey_results' (dict)


In [10]:
survey_scores = pd.DataFrame({
    'subject_id': survey_results.keys(),
    'average_score': [details['average_score'] for details in survey_results.values()],
    'total_score': [details['total_score'] for details in survey_results.values()]
})

# Loop through each subject to add question scores to the DataFrame
for subject_id, details in survey_results.items():
    df = details['data']
    for question in df['text']:
        survey_scores.loc[survey_scores['subject_id'] == subject_id, question] = details[question]

survey_scores = survey_scores.sort_values(by='subject_id')
%store survey_scores

Stored 'survey_scores' (DataFrame)
